# AirSplatMap Camera Testing

Test camera connection and pose/depth estimation in real-time.

## Features
- Camera connection testing
- Live pose estimation with trajectory visualization
- Depth estimation preview
- Compare multiple pose estimators side-by-side

## Requirements
- Webcam or RealSense camera
- OpenCV (`pip install opencv-python`)
- matplotlib for visualization

In [ ]:
import sys
from pathlib import Path
import numpy as np
import cv2
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

# Add project root
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
print(f"OpenCV version: {cv2.__version__}")

## 1. Test Camera Connection

In [ ]:
def test_camera(camera_id=0):
    """Test if camera is accessible."""
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"❌ Cannot open camera {camera_id}")
        return False
    
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        print(f"✅ Camera {camera_id} working! Frame size: {frame.shape}")
        return True
    else:
        print(f"❌ Camera {camera_id} cannot capture frames")
        return False

# Test default camera
test_camera(0)

In [ ]:
def capture_single_frame(camera_id=0):
    """Capture and display a single frame."""
    cap = cv2.VideoCapture(camera_id)
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        # Convert BGR to RGB for display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 6))
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.title(f'Camera {camera_id} - {frame.shape[1]}x{frame.shape[0]}')
        plt.show()
        return frame_rgb
    else:
        print("Failed to capture frame")
        return None

frame = capture_single_frame()

## 2. Test Pose Estimation

In [ ]:
from src.pose import get_pose_estimator, list_pose_estimators

print("Available pose estimators:")
for name, info in list_pose_estimators().items():
    gpu = "(GPU)" if info.get('gpu') else ""
    print(f"  - {name}: {info['description']} {gpu}")

In [ ]:
# Initialize pose estimator
POSE_METHOD = 'orb'  # Change to 'sift', 'loftr', 'superpoint' etc.

pose_estimator = get_pose_estimator(POSE_METHOD)

# Set camera intrinsics (adjust for your camera)
# Default webcam approximation for 640x480
pose_estimator.set_intrinsics_from_dict({
    'fx': 525.0,
    'fy': 525.0,
    'cx': 320.0,
    'cy': 240.0,
})

print(f"Initialized {POSE_METHOD} pose estimator")

In [ ]:
def test_pose_on_frames(n_frames=30, camera_id=0):
    """Capture frames and test pose estimation."""
    cap = cv2.VideoCapture(camera_id)
    
    positions = []
    fps_list = []
    
    print(f"Capturing {n_frames} frames...")
    print("Move the camera slowly to test tracking.")
    
    for i in range(n_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to RGB
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Estimate pose
        t0 = time.time()
        result = pose_estimator.estimate(rgb)
        dt = time.time() - t0
        
        positions.append(result.pose[:3, 3].copy())
        fps_list.append(1.0 / dt if dt > 0 else 0)
        
        if (i + 1) % 10 == 0:
            pos = result.pose[:3, 3]
            print(f"Frame {i+1}: pos=[{pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f}], "
                  f"inliers={result.num_inliers}, FPS={fps_list[-1]:.1f}")
    
    cap.release()
    
    return np.array(positions), fps_list

positions, fps_list = test_pose_on_frames(30)

In [ ]:
# Visualize trajectory
import matplotlib.pyplot as plt

if len(positions) > 1:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Trajectory (XZ plane)
    axes[0].plot(positions[:, 0], positions[:, 2], 'b-o', markersize=3)
    axes[0].plot(positions[0, 0], positions[0, 2], 'go', markersize=10, label='Start')
    axes[0].plot(positions[-1, 0], positions[-1, 2], 'ro', markersize=10, label='End')
    axes[0].set_xlabel('X (m)')
    axes[0].set_ylabel('Z (m)')
    axes[0].set_title('Camera Trajectory (Top-Down)')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].axis('equal')
    
    # FPS over time
    axes[1].plot(fps_list, 'g-')
    axes[1].axhline(y=np.mean(fps_list), color='r', linestyle='--', label=f'Avg: {np.mean(fps_list):.1f}')
    axes[1].set_xlabel('Frame')
    axes[1].set_ylabel('FPS')
    axes[1].set_title(f'Processing Speed ({POSE_METHOD})')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("Not enough frames captured")

## 3. Test Depth Estimation

In [ ]:
# Test depth estimation on a single frame
try:
    from benchmarks.depth.benchmark_depth import get_depth_estimator, list_depth_estimators
    
    print("Available depth estimators:")
    for name, info in list_depth_estimators().items():
        print(f"  - {name}: {info['description']}")
except ImportError as e:
    print(f"Depth estimators not available: {e}")

In [ ]:
DEPTH_METHOD = 'midas_small'  # Fast option

try:
    depth_estimator = get_depth_estimator(DEPTH_METHOD)
    print(f"Initialized {DEPTH_METHOD} depth estimator")
    
    # Capture and process
    if frame is not None:
        print("Estimating depth...")
        t0 = time.time()
        depth = depth_estimator.estimate(frame)
        dt = time.time() - t0
        print(f"Done in {dt:.2f}s ({1/dt:.1f} FPS)")
        
        # Visualize
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        axes[0].imshow(frame)
        axes[0].set_title('RGB Input')
        axes[0].axis('off')
        
        im = axes[1].imshow(depth, cmap='plasma')
        axes[1].set_title(f'Estimated Depth ({DEPTH_METHOD})')
        axes[1].axis('off')
        plt.colorbar(im, ax=axes[1], fraction=0.046)
        
        plt.tight_layout()
        plt.show()
except Exception as e:
    print(f"Depth estimation failed: {e}")

## 4. Compare Pose Estimators

In [ ]:
def compare_pose_estimators(methods=['orb', 'sift', 'robust_flow'], n_frames=20, camera_id=0):
    """Compare multiple pose estimators on live camera."""
    
    # Capture frames first
    cap = cv2.VideoCapture(camera_id)
    frames = []
    
    print(f"Capturing {n_frames} frames...")
    for i in range(n_frames):
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        time.sleep(0.05)  # ~20 FPS capture
    cap.release()
    
    print(f"Captured {len(frames)} frames")
    
    # Test each method
    results = {}
    
    for method in methods:
        print(f"\nTesting {method}...")
        try:
            estimator = get_pose_estimator(method)
            estimator.set_intrinsics_from_dict({
                'fx': 525.0, 'fy': 525.0, 'cx': 320.0, 'cy': 240.0
            })
            
            positions = []
            times = []
            
            for frame in frames:
                t0 = time.time()
                result = estimator.estimate(frame)
                times.append(time.time() - t0)
                positions.append(result.pose[:3, 3].copy())
            
            results[method] = {
                'positions': np.array(positions),
                'avg_fps': 1.0 / np.mean(times),
                'times': times,
            }
            print(f"  {method}: Avg FPS = {results[method]['avg_fps']:.1f}")
            
        except Exception as e:
            print(f"  {method}: FAILED - {e}")
    
    return results

# Run comparison
comparison = compare_pose_estimators(['orb', 'sift', 'robust_flow'], n_frames=20)

In [ ]:
# Visualize comparison
if comparison:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Trajectories
    for method, data in comparison.items():
        pos = data['positions']
        axes[0].plot(pos[:, 0], pos[:, 2], '-o', markersize=2, label=method)
    
    axes[0].set_xlabel('X (m)')
    axes[0].set_ylabel('Z (m)')
    axes[0].set_title('Trajectory Comparison')
    axes[0].legend()
    axes[0].grid(True)
    
    # FPS comparison
    methods = list(comparison.keys())
    fps_values = [comparison[m]['avg_fps'] for m in methods]
    
    axes[1].bar(methods, fps_values, color=['#1f77b4', '#ff7f0e', '#2ca02c'][:len(methods)])
    axes[1].set_ylabel('FPS')
    axes[1].set_title('Processing Speed Comparison')
    
    for i, v in enumerate(fps_values):
        axes[1].text(i, v + 1, f'{v:.1f}', ha='center')
    
    plt.tight_layout()
    plt.show()

## 5. Interactive Live View (Optional)

For continuous live streaming, it's better to use the web dashboard:

```bash
# Start RealSense server
python scripts/tools/realsense_server.py --port 8554

# Start dashboard
python scripts/web_dashboard.py
```

Then open http://localhost:9002 in your browser.

In [ ]:
print("To run live 3DGS with camera:")
print("")
print("1. Start RealSense server:")
print("   python scripts/tools/realsense_server.py --port 8554")
print("")
print("2. Start dashboard:")
print("   python scripts/web_dashboard.py")
print("")
print("3. Open browser:")
print("   http://localhost:9002")